# Tempt with PCA

In [24]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
#import sys
#my_path = r'Scrivania/Machine_Learning/Project_2/Project2'
#sys.path.insert(0,my_path + r'/code/ilaria')

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
%%bash
chmod u+x ./build_vocab.sh
chmod u+x ./cut_vocab.sh

In [26]:
from pickle_vocab import main as pickle_vocab
from cooc import main as cooc

os.system('build_vocab.sh')
os.system('cut_vocab.sh')

pickle_vocab()
cooc()# cooccurrence matrix

pickle_vocab.py executed
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
summing duplicates ... (this can take a while)
summing duplicates FINISHED !


In [27]:
from glove_solution import main as glove

glove()


loading cooccurrence matrix
6546685 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [28]:
X = np.load('embeddings.npy')
print(X.shape)

(21750, 25)


In [29]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [30]:
# build positive tweet feature set
X_pos = []

with open('train_pos.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_pos.append(list(np.mean(X_tweet, 0)))
        
    X_pos = np.array(X_pos)

In [31]:
# build negative tweet feature set
X_neg = []

with open('train_neg.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_neg.append(list(np.mean(X_tweet, 0)))
        
    X_neg = np.array(X_neg)     

In [32]:
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [33]:
print(X_pos.shape)
print(X_neg.shape)

(99952, 25)
(99989, 25)


In [34]:
X_pos_neg = np.concatenate([X_pos, X_neg])
y_pos_neg = np.concatenate([y_pos, y_neg])

X_pos_neg.shape

(199941, 25)

# Training and testing set

In [35]:
ind = np.arange(y_pos_neg.shape[0])
np.random.shuffle(ind)

r = 0.7
ir = round(r*X_pos_neg.shape[0])

X_tr = X_pos_neg[ind[:ir],:]
X_te = X_pos_neg[ind[ir:],:]
y_tr = y_pos_neg[ind[:ir]]
y_te = y_pos_neg[ind[ir:]]


n_samples, n_features = np.shape(X_pos_neg)

# PCA

In [36]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [37]:
std_scale = preprocessing.StandardScaler().fit(X_pos_neg)
X_pos_neg_norm = std_scale.transform(X_pos_neg)

pca = PCA(n_components=min(n_samples, n_features))

In [ ]:
pca.fit(X_pos_neg_norm)
Xposneg_PCA=pca.transform(X_pos_neg_norm)

print(X_pos_neg.shape)
print(Xposneg_PCA.shape)

(199941, 25)
(199941, 25)


# SVM

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(Xposneg_PCA, y_pos_neg) 

# Submission

In [ ]:
X_test = []
ids_test = []

with open('test_data.txt') as f:
    
    for idx, line in enumerate(f):
        #print(line)
        id_ = line.lstrip().split(',')[0]
        ids_test.append(id_)
        tweet = line.lstrip().split(',')[1:]
        #print(id_)
        #print(tweet)
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_test.append(list(np.mean(X_tweet, 0)))
        else:
            print(tweet)
            print(line)
            X_test.append(np.random.rand(20))    
            
        
print(len(X_test))        
X_test = np.array(X_test)
ids_test = np.array(ids_test)

In [ ]:
X_test= std_scale.transform(X_test)

pca.fit(X_test)
X_test=pca.transform(X_test)

y_pred = clf.predict(X_test)

In [ ]:
from create_csv_submission import create_csv_submission

name = 'sub512'
create_csv_submission(ids_test, y_pred, name)